Script to use the finetuned model to classify the first three images of all boats

In [2]:
from transformers import pipeline
import pandas as pd
from IPython.display import Image
from PIL import Image


df = pd.read_csv("imageDF.csv", sep=";", header = 0)

# cast all columns of df to string
df = df.astype(str)

replace the number in the column name to get the respective image per boat

In [ ]:
image_urls = df["image 1"]

#convert image_urls to list
image_urls = image_urls.tolist()

# get indices of the elements in image_urls which do not contain "https"
indices = [i for i, s in enumerate(image_urls) if "https" not in s]
 
# remove the elements in image_urls which do not contain "https"
for index in sorted(indices, reverse=True):
    del image_urls[index]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
from transformers import pipeline
predictor = pipeline("image-classification", model="henry-heppe/finetuned_convnextv2", image_processor="facebook/convnextv2-tiny-1k-224")

In [ ]:
from tqdm import tqdm
from PIL import UnidentifiedImageError

image_class = []
for url in tqdm(image_urls):
    try:
        out = predictor(url)
        image_class.append(out[0]["label"])
    except UnidentifiedImageError:
        image_class.append("Error")
        continue

In [60]:
# add NaNs at indices where there are no images (based on the indices variable)
for index in indices:
    image_class.insert(index, "")

In [ ]:
# add image class to df as column
iteration = str(1)
df = df.head(24)
df["image class" + iteration] = image_class

In [78]:
# save the dataframe to a csv file
df.to_csv("imageDF1.csv", sep=";", index=False)

In [ ]:
# in google colab download the file
#from google.colab import files
#files.download("/content/imageDF1.csv")